# Create SRA submission sheet
This Python Jupyter notebook creates a BioSample submission sheet for the Sequence Read Archive:

First, import Python modules:

In [1]:
import datetime
import glob
import itertools
import os

import natsort

import pandas as pd

import yaml

Display full columns of data frames:

In [2]:
pd.set_option('display.max_colwidth', None)

Read the configuration for the uploads:

In [3]:
with open('upload_config.yaml') as f:
    config = yaml.safe_load(f)

Read the Illumina runs to submit:

In [4]:
print(f"Reading Illumina runs from {config['barcode_runs']}")

illumina_runs = pd.read_csv(config['barcode_runs'])

print('Here are the first few entries in the Illumina runs:')
illumina_runs.head()

Reading Illumina runs from ../data/barcode_runs_SRA-upload.csv
Here are the first few entries in the Illumina runs:


,library,sample,date,experiment,antibody,concentration,concentration_units,group,selection,frac_escape,cells_sorted,R1
0,lib1,expt_74-78-none-0-reference,201109,expt_74-78,none,0,ng_per_mL,Vir,reference,NaN,NaN,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt74_78_lib1_ref*.fastq.gz; /shared/ngs/illumina/tstarr/201116_D00300_1113_BHJKKFBCX3/Unaligned/Project_tstarr/expt74_78_lib1_ref*.fastq.gz
1,lib2,expt_74-78-none-0-reference,201109,expt_74-78,none,0,ng_per_mL,Vir,reference,NaN,NaN,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt74_78_lib2_ref*.fastq.gz; /shared/ngs/illumina/tstarr/201116_D00300_1113_BHJKKFBCX3/Unaligned/Project_tstarr/expt74_78_lib2_ref*.fastq.gz
2,lib1,expt_68-73-none-0-reference,201106,expt_68-73,none,0,ng_per_mL,Vir,reference,NaN,NaN,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt68_73_lib1_ref*.fastq.gz; /shared/ngs/illumina/tstarr/201116_D00300_1113_BHJKKFBCX3/Unaligned/Project_tstarr/expt68_73_lib1_ref*.fastq.gz
3,lib2,expt_68-73-none-0-reference,201106,expt_68-73,none,0,ng_per_mL,Vir,reference,NaN,NaN,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt68_73_lib2_ref*.fastq.gz; /shared/ngs/illumina/tstarr/201116_D00300_1113_BHJKKFBCX3/Unaligned/Project_tstarr/expt68_73_lib2_ref*.fastq.gz
4,lib1,expt_79-81-none-0-reference,210113,expt_79-81,none,0,ng_per_mL,Vir,reference,NaN,NaN,/shared/ngs/illumina/tstarr/210115_D00300_1154_BHK3HCBCX3/Unaligned/Project_tstarr/expt_79-81_lib1_ref*.fastq.gz


Next make submission entries for the barcode runs, initially in "tidy" format with one FASTQ file per row:

In [5]:
submissions_tidy = (
    illumina_runs
    .assign(
        biosample_accession=lambda x: config['biosample_accession'],
        library_ID=lambda x: x['library'] + '_' + x['sample'],
        title=lambda x: 'SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for ' + x['sample'],
        library_strategy='AMPLICON',
        library_source='SYNTHETIC',
        library_selection='PCR',
        library_layout='single',
        platform='ILLUMINA',
        instrument_model='Illumina HiSeq 2500',
        design_description='PCR of barcodes from RBD variants',
        filetype='fastq',
        filename_fullpath=lambda x: x['R1'].str.split(';')
                                    .map(lambda flist: list(itertools.chain.from_iterable(glob.glob(f) for f in flist))),       
        )
    .explode('filename_fullpath')
    .assign(filename_fullpath=lambda x: x['filename_fullpath'].str.strip(),
            filename=lambda x: x['filename_fullpath'].map(os.path.basename))
    .drop(columns=illumina_runs.columns)
    .reset_index(drop=True)
    )

assert submissions_tidy['filename_fullpath'].map(os.path.isfile).all()

print('Here are the first few submission entries in tidy format:')
submissions_tidy.head()

Here are the first few submission entries in tidy format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename_fullpath,filename
0,SAMN18315604,lib1_expt_74-78-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_74-78-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt74_78_lib1_ref3_S155_R1_001.fastq.gz,expt74_78_lib1_ref3_S155_R1_001.fastq.gz
1,SAMN18315604,lib1_expt_74-78-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_74-78-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt74_78_lib1_ref2_S154_R1_001.fastq.gz,expt74_78_lib1_ref2_S154_R1_001.fastq.gz
2,SAMN18315604,lib1_expt_74-78-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_74-78-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt74_78_lib1_ref4_S156_R1_001.fastq.gz,expt74_78_lib1_ref4_S156_R1_001.fastq.gz
3,SAMN18315604,lib1_expt_74-78-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_74-78-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt74_78_lib1_ref1_S153_R1_001.fastq.gz,expt74_78_lib1_ref1_S153_R1_001.fastq.gz
4,SAMN18315604,lib2_expt_74-78-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_74-78-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,/shared/ngs/illumina/tstarr/201123_D00300_1119_BHJKJTBCX3/Unaligned/Project_tstarr/expt74_78_lib2_ref1_S157_R1_001.fastq.gz,expt74_78_lib2_ref1_S157_R1_001.fastq.gz


For the actual submission, we need a "wide" data frame that for each unique `sample_name` / `library_ID` gives all of the files each in different columns.
These should be files without the full path.

First, look at how many files there are for each sample / library:

In [6]:
(submissions_tidy
 .groupby(['biosample_accession', 'library_ID'])
 .aggregate(n_files=pd.NamedAgg('filename_fullpath', 'count'))
 .sort_values('n_files', ascending=False)
 .reset_index()
 )

,biosample_accession,library_ID,n_files
0,SAMN18315604,lib1_expt_68-73-none-0-reference,4
1,SAMN18315604,lib1_expt_74-78-none-0-reference,4
2,SAMN18315604,lib2_expt_79-81-none-0-reference,4
3,SAMN18315604,lib2_expt_74-78-none-0-reference,4
4,SAMN18315604,lib1_expt_79-81-none-0-reference,4
5,SAMN18315604,lib2_expt_68-73-none-0-reference,4
6,SAMN18315604,lib1_expt_73-S304-46-escape,1
7,SAMN18315604,lib1_expt_72-S2X58-18-escape,1
8,SAMN18315604,lib2_expt_80-S2H13-56-escape,1
9,SAMN18315604,lib2_expt_79-S2H97-58-escape,1


Now make the wide submission data frame.
Note we keep only the filename column with the path lacking the full directory information:

In [7]:
submissions_wide = (
    submissions_tidy
    .assign(
        filename_count=lambda x: x.groupby(['biosample_accession', 'library_ID'])['filename'].cumcount() + 1,
        filename_col=lambda x: 'filename' + x['filename_count'].map(lambda c: str(c) if c > 1 else '')
        )
    .pivot(
        index='library_ID',
        columns='filename_col',
        values='filename',
        )
    )

submissions_wide = (
    submissions_tidy
    .drop(columns=['filename_fullpath', 'filename'])
    .drop_duplicates()
    .merge(submissions_wide[natsort.natsorted(submissions_wide.columns)],
           on='library_ID',
           validate='one_to_one',
           )
    )

print('Here are the first few submission entries in wide format:')
submissions_wide.head()

Here are the first few submission entries in wide format:


,biosample_accession,library_ID,title,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,filetype,filename,filename2,filename3,filename4
0,SAMN18315604,lib1_expt_74-78-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_74-78-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt74_78_lib1_ref3_S155_R1_001.fastq.gz,expt74_78_lib1_ref2_S154_R1_001.fastq.gz,expt74_78_lib1_ref4_S156_R1_001.fastq.gz,expt74_78_lib1_ref1_S153_R1_001.fastq.gz
1,SAMN18315604,lib2_expt_74-78-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_74-78-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt74_78_lib2_ref1_S157_R1_001.fastq.gz,expt74_78_lib2_ref4_S160_R1_001.fastq.gz,expt74_78_lib2_ref2_S158_R1_001.fastq.gz,expt74_78_lib2_ref3_S159_R1_001.fastq.gz
2,SAMN18315604,lib1_expt_68-73-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_68-73-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt68_73_lib1_ref4_S114_R1_001.fastq.gz,expt68_73_lib1_ref2_S112_R1_001.fastq.gz,expt68_73_lib1_ref1_S111_R1_001.fastq.gz,expt68_73_lib1_ref3_S113_R1_001.fastq.gz
3,SAMN18315604,lib2_expt_68-73-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_68-73-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt68_73_lib2_ref4_S118_R1_001.fastq.gz,expt68_73_lib2_ref1_S115_R1_001.fastq.gz,expt68_73_lib2_ref3_S117_R1_001.fastq.gz,expt68_73_lib2_ref2_S116_R1_001.fastq.gz
4,SAMN18315604,lib1_expt_79-81-none-0-reference,SARS-CoV-2 RBD mutational antig Illumina barcode sequencing for expt_79-81-none-0-reference,AMPLICON,SYNTHETIC,PCR,single,ILLUMINA,Illumina HiSeq 2500,PCR of barcodes from RBD variants,fastq,expt_79-81_lib1_ref3_S79_R1_001.fastq.gz,expt_79-81_lib1_ref1_S77_R1_001.fastq.gz,expt_79-81_lib1_ref4_S80_R1_001.fastq.gz,expt_79-81_lib1_ref2_S78_R1_001.fastq.gz


Now write the wide submissions data frame to a `*.tsv` file that can be used for uploading the SRA submission website:

In [8]:
submissions_spreadsheet = 'SRA_submission_spreadsheet.tsv'

submissions_wide.to_csv(submissions_spreadsheet, sep='\t', index=False)

We also want to write a file with all of the FASTQ files that are in the submission spreadsheet:

In [9]:
fastq_file_list = 'FASTQs_to_upload.csv'

print(f"Writing list of all the FASTQ files to upload to {fastq_file_list}")

(submissions_tidy
 [['filename_fullpath', 'filename']]
 .to_csv(fastq_file_list, index=False)
 )

Writing list of all the FASTQ files to upload to FASTQs_to_upload.csv
